# Image Quality Control Demo

This notebook demonstrates the image quality control utilities for manual and automatic QC flagging.

Run each cell in order to see the complete workflow.

In [6]:
# Setup - Import required modules
import sys
import os
import pandas as pd
from pathlib import Path

# Determine the correct path to the image_quality_qc_utils module
current_dir = Path.cwd()
print(f"Current directory: {current_dir}")

# Look for the utils/image_quality_qc_utils directory
utils_qc_path = None

# Check if we're in the segmentation_sandbox directory
if current_dir.name == "segmentation_sandbox":
    utils_qc_path = current_dir / "utils" / "image_quality_qc_utils"
# Check if we're in the test directory
elif current_dir.name == "test":
    utils_qc_path = current_dir.parent
# Check if we're somewhere else in the project
else:
    # Try to find segmentation_sandbox in the path
    for parent in current_dir.parents:
        if parent.name == "segmentation_sandbox":
            utils_qc_path = parent / "utils" / "image_quality_qc_utils"
            break

if utils_qc_path and utils_qc_path.exists():
    sys.path.insert(0, str(utils_qc_path))
    print(f"Added to path: {utils_qc_path}")
else:
    print("ERROR: Could not find image_quality_qc_utils directory")
    print(f"Current directory: {current_dir}")
    print("Please run this notebook from the segmentation_sandbox directory or test directory")

# Import directly from image_quality_qc_utils
from image_quality_qc_utils import (
    load_qc_data, save_qc_data, flag_qc, remove_qc, 
    get_qc_summary, get_flagged_images, get_unflagged_images,
    get_images_by_flag, get_images_by_annotator, QC_FLAGS,
    manual_qc, auto_qc
)

print("Successfully imported QC utilities!")
print(f"Available QC flags: {list(QC_FLAGS.keys())}")

Current directory: /net/trapnell/vol1/home/mdcolon/proj/morphseq/segmentation_sandbox
Parent directory: /net/trapnell/vol1/home/mdcolon/proj/morphseq


ModuleNotFoundError: No module named 'image_quality_qc_utils'

## Setup Test Data

Create a test directory and some sample QC data.

In [ ]:
# Create test data directory
test_data_dir = current_dir / "notebook_test_data"
test_data_dir.mkdir(exist_ok=True)

print(f"Test data directory: {test_data_dir}")

# Load or create QC data
qc_df = load_qc_data(test_data_dir)
print(f"Initial QC records: {len(qc_df)}")

## Add Sample Images

In [ ]:
# Add some sample images if not already present
sample_images = [
    {"experiment_id": "20241215", "video_id": "20241215_A01", "image_id": "20241215_A01_t001"},
    {"experiment_id": "20241215", "video_id": "20241215_A01", "image_id": "20241215_A01_t002"},
    {"experiment_id": "20241215", "video_id": "20241215_B01", "image_id": "20241215_B01_t001"},
    {"experiment_id": "20241216", "video_id": "20241216_A01", "image_id": "20241216_A01_t001"},
]

for img in sample_images:
    if img["image_id"] not in qc_df["image_id"].values:
        new_row = pd.DataFrame([{
            "experiment_id": img["experiment_id"],
            "video_id": img["video_id"], 
            "image_id": img["image_id"],
            "qc_flag": None,
            "notes": None,
            "annotator": None
        }])
        qc_df = pd.concat([qc_df, new_row], ignore_index=True)

save_qc_data(qc_df, test_data_dir)
print(f"Total records: {len(qc_df)}")
display(qc_df.head())

## Manual QC Flagging

In [ ]:
# Flag an image manually
qc_df = flag_qc(
    data_dir=test_data_dir,
    image_ids=["20241215_A01_t001"],
    qc_flag="BLUR",
    annotator="mcolon",
    notes="Manual inspection - image appears blurry",
    overwrite=True
)

print("Flagged image as BLUR")

# Show flagged images
flagged = qc_df[qc_df["qc_flag"].notna()]
display(flagged)

## Automatic QC Flagging

In [ ]:
# Simulate automatic flagging
qc_df = flag_qc(
    data_dir=test_data_dir,
    image_ids=["20241215_A01_t002"],
    qc_flag="PASS",
    annotator="automatic",
    notes="Automatic: Quality metrics within normal range",
    overwrite=True
)

print("Automatically flagged image as PASS")

# Show all flagged images
flagged = qc_df[qc_df["qc_flag"].notna()]
display(flagged)

## Analyze QC Data

In [ ]:
# Get QC summary
summary = get_qc_summary(test_data_dir)
print("QC Summary:")
for key, value in summary.items():
    print(f"  {key}: {value}")

# Use analysis functions
flagged_images = get_flagged_images(qc_df)
unflagged_images = get_unflagged_images(qc_df)

print(f"\nFlagged images: {len(flagged_images)}")
print(f"Unflagged images: {len(unflagged_images)}")

# Get images by specific criteria
blur_images = get_images_by_flag(qc_df, "BLUR")
manual_images = get_images_by_annotator(qc_df, "mcolon")

print(f"\nBLUR flagged images: {blur_images}")
print(f"Manually flagged images: {manual_images}")

## Final Results

In [ ]:
# Show final QC data
print("Final QC data:")
display(qc_df)

# Save to CSV for inspection
output_file = current_dir / "notebook_qc_results.csv"
qc_df.to_csv(output_file, index=False)
print(f"\nResults saved to: {output_file}")